# Data Scraping (Yelp)

In this notebook, i will be scraping restaurant data from Yelp

In [1]:
import pandas as pd 
import re
from scrapingbee import ScrapingBeeClient
from bs4 import BeautifulSoup
import requests
import time

### Set up requests library to retrieve data from target website (Single Page for testing)

In [2]:
client = ScrapingBeeClient(api_key = '9LF9S52L1EX49H34GBV3JX5NEUKDQJGN8K8KQOD7NKTV57R5YSBKAKIN3GDZS497TYB137CFNF9X25W0')

#indicate base url
url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Singapore&attrs=RestaurantsPriceRange2.1'
res = requests.get(url)

In [3]:
res.status_code

200

In [4]:
res.text # returns html

'<!DOCTYPE html><html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/\x08no-js\x08/,"js");</script><meta http-equiv="Content-Type" content="text/html; charset=UTF-8" /><meta http-equiv="Content-Language" content="en-US" /><meta name="viewport" content="width=device-width, initial-scale=1, shrink-to-fit=no"><link rel="mask-icon" sizes="any" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" content="#FF1A1A"><link rel="shortcut icon" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico"><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>\n            window.yelp = window.yelp || {};\

### Create a BeautifulSoup object

In [5]:
soup = BeautifulSoup(res.text, 'lxml')  # parse html for python
soup

<!DOCTYPE html>
<html lang="en-US" prefix="og: http://ogp.me/ns#" style="margin: 0;padding: 0; border: 0; font-size: 100%; font: inherit; vertical-align: baseline;"><head><script>document.documentElement.className=document.documentElement.className.replace(/no-js/,"js");</script><meta content="text/html; charset=utf-8" http-equiv="Content-Type"/><meta content="en-US" http-equiv="Content-Language"/><meta content="width=device-width, initial-scale=1, shrink-to-fit=no" name="viewport"/><link content="#FF1A1A" href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/b2bb2fb0ec9c/assets/img/logos/yelp_burst.svg" rel="mask-icon" sizes="any"/><link href="https://s3-media0.fl.yelpcdn.com/assets/srv0/yelp_large_assets/dcfe403147fc/assets/img/logos/favicon.ico" rel="shortcut icon"/><script> window.ga=window.ga||function(){(ga.q=ga.q||[]).push(arguments)};ga.l=+new Date;window.ygaPageStartTime=new Date().getTime();</script><script>
            window.yelp = window.yelp || {};
        

Testing of code to scrape restaurant name

In [6]:
restaurant = soup.find('div', class_="businessName__09f24__HG_pC yelp-emotion-ohs7lg")

In [7]:
restaurant.text

'1.\xa0Sin Heng Kee Porridge'

## Scraping of first result in page (testing)

Scraping of restaurant name, rating, cuisine, price range and location

In [8]:
listing = soup.find('div', class_ = 'container__09f24__FeTO6 hoverable__09f24___UXLO yelp-emotion-way87j')
for item in listing:
    name = item.find('div', class_ = "businessName__09f24__HG_pC yelp-emotion-ohs7lg").text
    rating = item.find('span', class_ = "yelp-emotion-x4y0dd").text
    cuisine = item.find('button', class_ = 'yelp-emotion-10d2rmz').text
    pricerange = item.find('div', class_= 'yelp-emotion-17hd2hl')
    price = pricerange.find('span', class_= 'priceRange__09f24__ZgJXy yelp-emotion-oyrex3').text
    location = pricerange.find('span', class_= 'yelp-emotion-v293gj').text


print(f'name: {name}\nrating:{rating}\ncuisine: {cuisine}\nprice: {price}\nlocation: {location}')


name: 1. Sin Heng Kee Porridge
rating:4.8 
cuisine: Singaporean
price: $
location: Hougang


Scraping all restaurants in a Single Page and storing in dictionary for conversion to dataframe later on

In [10]:
listings = soup.find_all('div', class_='container__09f24__FeTO6 hoverable__09f24___UXLO yelp-emotion-way87j')


#instantiate a new restaurants list
restaurants_list = []

# Set counter for iterating thru multiple pages in the website
count = 1

    
for item in listings:
    #instantiate a new restaurants dictionary 
    restaurants_dict = {}
    name = item.find('div', class_="businessName__09f24__HG_pC yelp-emotion-ohs7lg").text
    rating = item.find('span', class_="yelp-emotion-x4y0dd").text
    cuisine_tags = item.find_all('button', class_='yelp-emotion-10d2rmz')
    cuisine = [tag.text for tag in cuisine_tags]
    cuisines = ', '.join(cuisine)
    pricerange = item.find('div', class_='yelp-emotion-17hd2hl')
    
    if pricerange:
        price = pricerange.find('span', class_='priceRange__09f24__ZgJXy yelp-emotion-oyrex3').text
        location = pricerange.find('span', class_='yelp-emotion-v293gj').text
    else:
        price = "Not available"
        location = "Not available"

    print(f'name: {name}\nrating: {rating}\ncuisine: {cuisines}\nprice: {price}\nlocation: {location}')
    print('---')

    restaurants_dict['restaurant number'] = count 
    restaurants_dict['name'] = name
    restaurants_dict['rating'] = rating 
    restaurants_dict['cuisine'] = cuisines
    restaurants_dict['price'] = price 
    restaurants_dict['location'] = location

    #append restaurant to restaurants list 
    restaurants_list.append(restaurants_dict)
    
    
    print(restaurants_dict)
    print('------------------------------')
    count += 1


name: 1. Sin Heng Kee Porridge
rating: 4.8 
cuisine: Singaporean, Congee
price: $
location: Hougang
---
{'restaurant number': 1, 'name': '1.\xa0Sin Heng Kee Porridge', 'rating': '4.8 ', 'cuisine': 'Singaporean, Congee', 'price': '$', 'location': 'Hougang'}
------------------------------
name: 2. French Ladle
rating: 4.8 
cuisine: French
price: $
location: Ulu Pandan
---
{'restaurant number': 2, 'name': '2.\xa0French Ladle', 'rating': '4.8 ', 'cuisine': 'French', 'price': '$', 'location': 'Ulu Pandan'}
------------------------------
name: 3. Hong Kong Style Chee Cheong Fun
rating: 4.6 
cuisine: Chinese
price: $
location: Farrer Park
---
{'restaurant number': 3, 'name': '3.\xa0Hong Kong Style Chee Cheong Fun', 'rating': '4.6 ', 'cuisine': 'Chinese', 'price': '$', 'location': 'Farrer Park'}
------------------------------
name: 4. Tian Tian Hainanese Chicken Rice
rating: 3.9 
cuisine: Hainan, Chicken Shop
price: $
location: Tanjong Pagar
---
{'restaurant number': 4, 'name': '4.\xa0Tian Tia

Storing Restaurants List in a dataframe

In [11]:
df = pd.DataFrame(restaurants_list)
df

,restaurant number,name,rating,cuisine,price,location
0,1,1. Sin Heng Kee Porridge,4.8,"Singaporean, Congee",$,Hougang
1,2,2. French Ladle,4.8,French,$,Ulu Pandan
2,3,3. Hong Kong Style Chee Cheong Fun,4.6,Chinese,$,Farrer Park
3,4,4. Tian Tian Hainanese Chicken Rice,3.9,"Hainan, Chicken Shop",$,Tanjong Pagar
4,5,5. Song Fa Bak Kut Teh,4.2,"Singaporean, Chinese",$,Clarke Quay
5,6,6. Two Men Bagel House,4.5,"Bagels, Breakfast & Brunch, Sandwiches",$,Tanjong Pagar
6,7,7. Song Fa Bak Kut Teh,4.6,"Chinese, Singaporean",$,Chinatown
7,8,8. Tim Ho Wan,4.3,"Dim Sum, Cantonese",$,City Hall
8,9,9. Zhong Guo La Mian Xiao Long Bao,4.4,"Shanghainese, Dim Sum, Noodles",$,Chinatown
9,10,10. Wee Nam Kee Chicken Rice,4.3,"Chicken Shop, Hainan",$,Novena


## Scraping of multiple pages

In [12]:
base_url = 'https://www.yelp.com/search?find_desc=Restaurants&find_loc=Singapore&l=g%3A103.74664306640625%2C1.2111524670742526%2C103.95126342773438%2C1.417091829441639&attrs=RestaurantsPriceRange2.1%2CRestaurantsPriceRange2.2%2CRestaurantsPriceRange2.3' 

#instantiate a new restaurants list
restaurants_list = []

# Set counter for iterating thru multiple pages in the website
count = 1

total_pages = 24

for page_number in range(1, total_pages + 1):  # Scraping first 24 pages
    url = f'{base_url}&start={page_number * 10}'  # Assuming 10 results per page
    response = requests.get(url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.content, 'lxml')
        listings1 = soup.find_all('div', class_='container__09f24__FeTO6 hoverable__09f24___UXLO yelp-emotion-way87j')

        for item in listings1:
            #instantiate a new restaurants dictionary 
            restaurants_dict = {}
            name = item.find('div', class_="businessName__09f24__HG_pC yelp-emotion-ohs7lg").text
            rating = item.find('span', class_="yelp-emotion-x4y0dd").text
            cuisine_tags = item.find_all('button', class_='yelp-emotion-10d2rmz')
            cuisine = [tag.text for tag in cuisine_tags]
            cuisines = ', '.join(cuisine)
            pricerange = item.find('div', class_='yelp-emotion-17hd2hl')
            
            if pricerange:
                price_element = pricerange.find('span', class_='priceRange__09f24__ZgJXy yelp-emotion-oyrex3')
                price = price_element.text if price_element else "Not available"
                location_element = pricerange.find('span', class_='yelp-emotion-v293gj')
                location = location_element.text if location_element else "Not available"

                #price = pricerange.find('span', class_='priceRange__09f24__ZgJXy yelp-emotion-oyrex3').text
                #location = pricerange.find('span', class_='yelp-emotion-v293gj').text
            else:
                price = "Not available"
                location = "Not available"

            print(f'name: {name}\nrating: {rating}\ncuisine: {cuisines}\nprice: {price}\nlocation: {location}')
            print('---')

            restaurants_dict['restaurant number'] = count 
            restaurants_dict['name'] = name
            restaurants_dict['rating'] = rating 
            restaurants_dict['cuisine'] = cuisines
            restaurants_dict['price'] = price 
            restaurants_dict['location'] = location

            #append restaurant to restaurants list 
            restaurants_list.append(restaurants_dict)
            count +=1

        print(f"Scraped page {page_number} of {total_pages} (Total restaurants: {len(restaurants_list)})")
        time.sleep(1)

    else:
        print(f"Failed to fetch page {page_number}. Status code: {response.status_code}")

print("Scraping complete")

name: 11. The Garden @ Sentosa
rating: 5.0 
cuisine: Modern European
price: $$$
location: Sentosa
---
name: 12. Tian Tian Seafood Restaurant
rating: 4.7 
cuisine: Seafood, Chinese
price: $$
location: Tiong Bahru
---
name: 13. Song Fa Bak Kut Teh
rating: 4.6 
cuisine: Chinese, Singaporean
price: $
location: Chinatown
---
name: 14. Zhong Guo La Mian Xiao Long Bao
rating: 4.4 
cuisine: Shanghainese, Dim Sum, Noodles
price: $
location: Chinatown
---
name: 15. La Strada
rating: 4.4 
cuisine: Italian
price: $$$
location: Tanglin
---
name: 16. Wee Nam Kee Chicken Rice
rating: 4.3 
cuisine: Chicken Shop, Hainan
price: $
location: Novena
---
name: 17. Two Men Bagel House
rating: 4.5 
cuisine: Bagels, Breakfast & Brunch, Sandwiches
price: $
location: Tanjong Pagar
---
name: 18. Going Om
rating: 4.3 
cuisine: Bars, Cafes
price: $$
location: Arab Street
---
name: 19. Lang Nuong Viet Nam
rating: 4.4 
cuisine: Vietnamese, Barbeque
price: $$
location: Little India
---
name: 20. Sakurazaka
rating: 4.7

In [13]:
restaurants_list

[{'restaurant number': 1,
  'name': '11.\xa0The Garden @ Sentosa',
  'rating': '5.0 ',
  'cuisine': 'Modern European',
  'price': '$$$',
  'location': 'Sentosa'},
 {'restaurant number': 2,
  'name': '12.\xa0Tian Tian Seafood Restaurant',
  'rating': '4.7 ',
  'cuisine': 'Seafood, Chinese',
  'price': '$$',
  'location': 'Tiong Bahru'},
 {'restaurant number': 3,
  'name': '13.\xa0Song Fa Bak Kut Teh',
  'rating': '4.6 ',
  'cuisine': 'Chinese, Singaporean',
  'price': '$',
  'location': 'Chinatown'},
 {'restaurant number': 4,
  'name': '14.\xa0Zhong Guo La Mian Xiao Long Bao',
  'rating': '4.4 ',
  'cuisine': 'Shanghainese, Dim Sum, Noodles',
  'price': '$',
  'location': 'Chinatown'},
 {'restaurant number': 5,
  'name': '15.\xa0La Strada',
  'rating': '4.4 ',
  'cuisine': 'Italian',
  'price': '$$$',
  'location': 'Tanglin'},
 {'restaurant number': 6,
  'name': '16.\xa0Wee Nam Kee Chicken Rice',
  'rating': '4.3 ',
  'cuisine': 'Chicken Shop, Hainan',
  'price': '$',
  'location': 'Nov

Exporting scraped data

In [14]:
df = pd.DataFrame(restaurants_list)

In [15]:
df.head()


,restaurant number,name,rating,cuisine,price,location
0,1,11. The Garden @ Sentosa,5.0,Modern European,$$$,Sentosa
1,2,12. Tian Tian Seafood Restaurant,4.7,"Seafood, Chinese",$$,Tiong Bahru
2,3,13. Song Fa Bak Kut Teh,4.6,"Chinese, Singaporean",$,Chinatown
3,4,14. Zhong Guo La Mian Xiao Long Bao,4.4,"Shanghainese, Dim Sum, Noodles",$,Chinatown
4,5,15. La Strada,4.4,Italian,$$$,Tanglin


In [16]:
df.to_csv('../datasets/yelp.csv')

------------------

# 2. Scraping for Restaurants on Google

Results on Google Places was scraped externally on google sheets.